In [ ]:
import os

os.environ[
    'PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.3.1 --master local[2] pyspark-shell'

import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, concat, lit, col
from time import sleep

if __name__ == '__main__':
    mySpark = SparkSession.builder.appName("movieData").master("local").getOrCreate()
    df = mySpark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "localhost:9092") \
        .option("subscribe", "tweets") \
        .option("startingOffsets", "earliest") \
        .load()

    lines = df.selectExpr("CAST(value AS STRING)")

    # Split the lines into words
    words = (lines.select(
        explode(
            split(lines.value, " ")
        ).alias("word"))
    ) 
    
    movies = mySpark.read.format("com.mongodb.spark.sql.DefaultSource") \
          .option("uri","mongodb://localhost:27017/movieDatabase.movies").load()
    hashtack_list = []
    for h in movies.select('title_hashtack').toPandas()['title_hashtack']: # get all documents
        hashtack_list.append(h)
    
    words = words.filter(words.word.isin(hashtack_list))
    wordCounts = words.groupBy("word").count()
    #wordCounts.writeStream.queryName("words_count") \
    #    .format("memory").outputMode("complete") \
    #    .start()
    
    query = wordCounts \
              .select(concat(col("word"), lit(" "), col("count")).alias("value")) \
              .writeStream \
              .format("kafka") \
              .option("kafka.bootstrap.servers", "localhost:9092") \
              .option("checkpointLocation", "C:\\tmp\\cp6")\
              .option("topic", "tweets_analysis") \
              .outputMode("complete") \
              .start()
    query.awaitTermination()
    #for x in range(10):
    #    mySpark.sql("SELECT * FROM words_count").show()
    #    sleep(1)
